In [9]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import os

# Load data
data = pd.read_csv("merged_data.csv")
data = data.dropna()  # Remove all rows with NaN values


# Select features and target
X = data[['Pe_results', 'Comp_results', 'TAC_Reading']].values
y = data['Sober_classification'].values

# Normalize features
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Convert to tensors
X_tensor = torch.tensor(X, dtype=torch.float32)
y_tensor = torch.tensor(y, dtype=torch.float32).view(-1, 1)  # Reshape for binary classification

# Split into train and test
X_train, X_test, y_train, y_test = train_test_split(X_tensor, y_tensor, test_size=0.2, random_state=42)

# Move data to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
X_train, y_train, X_test, y_test = X_train.to(device), y_train.to(device), X_test.to(device), y_test.to(device)


# ===================== Fully Connected Neural Network (FNN) ===================== #
class FNN(nn.Module):
    def __init__(self):
        super(FNN, self).__init__()
        self.fc1 = nn.Linear(3, 16)
        self.fc2 = nn.Linear(16, 8)
        self.fc3 = nn.Linear(8, 1)  # Output layer

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)  # No sigmoid here (handled by BCEWithLogitsLoss)
        return x  # Keep raw logits

fnn_model = FNN().to(device)  # Initialize the FNN model


# ===================== Recurrent Neural Network (RNN) ===================== #
class RNN(nn.Module):
    def __init__(self, input_size=3, hidden_size=16, num_layers=1):
        super(RNN, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.rnn = nn.RNN(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, 1)

    def forward(self, x):
        x = x.unsqueeze(1)  # Add sequence dimension -> (batch_size, seq_length=1, input_size)
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        out, _ = self.rnn(x, h0)
        out = self.fc(out[:, -1, :])  # Take last time step output
        return out  # No Sigmoid

rnn_model = RNN().to(device)


# ===================== Long Short-Term Memory (LSTM) ===================== #
class LSTM(nn.Module):
    def __init__(self, input_size=3, hidden_size=16, num_layers=1):
        super(LSTM, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, 1)

    def forward(self, x):
        x = x.unsqueeze(1)  # (batch_size, seq_length=1, input_size)
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        out, _ = self.lstm(x, (h0, c0))
        out = self.fc(out[:, -1, :])
        return out  # No Sigmoid

lstm_model = LSTM().to(device)


# ===================== Convolutional Neural Network (CNN) ===================== #
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv1d(in_channels=1, out_channels=16, kernel_size=2, stride=1)
        self.pool = nn.MaxPool1d(kernel_size=2)
        self.fc1 = nn.Linear(16, 8)
        self.fc2 = nn.Linear(8, 1)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = x.unsqueeze(1)  # Add channel dimension -> (batch_size, 1, features)
        x = self.pool(self.relu(self.conv1(x)))
        x = x.view(x.size(0), -1)
        x = self.relu(self.fc1(x))
        x = self.fc2(x)  # No Sigmoid
        return x  # No Sigmoid

cnn_model = CNN().to(device)


# ===================== Training Function ===================== #

def train_model(model, X_train, y_train, X_test, y_test, model_name, epochs=100, lr=0.001):
    criterion = nn.BCEWithLogitsLoss()  # No sigmoid in model!
    optimizer = optim.Adam(model.parameters(), lr=lr)

    # Store losses for visualization
    train_losses = []
    test_losses = []

    for epoch in range(epochs):
        model.train()
        optimizer.zero_grad()
        outputs = model(X_train)
        loss = criterion(outputs, y_train)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)  # Prevents exploding gradients
        optimizer.step()

        # Evaluate on test set
        model.eval()
        with torch.no_grad():
            test_outputs = model(X_test)
            test_loss = criterion(test_outputs, y_test)

        train_losses.append(loss.item())
        test_losses.append(test_loss.item())

        if epoch % 10 == 0:
            print(f'[{model_name}] Epoch {epoch}/{epochs}, Loss: {loss.item():.4f}, Test Loss: {test_loss.item():.4f}')

    # Save losses to CSV
    loss_data = pd.DataFrame({
        'Epoch': list(range(epochs)),
        'Train Loss': train_losses,
        'Test Loss': test_losses,
        'Total Loss': np.array(train_losses) + np.array(test_losses)  # Sum of both
    })

    os.makedirs("logs", exist_ok=True)  # Create folder if it doesn't exist
    loss_data.to_csv(f'logs/{model_name}_losses.csv', index=False)

    # Save model
    os.makedirs("models", exist_ok=True)  # Create folder for models
    torch.save(model.state_dict(), f'models/{model_name}.pth')

    print(f'[{model_name}] Training complete. Model & loss data saved.')


# ===================== Training Models ===================== #
train_model(fnn_model, X_train, y_train, X_test, y_test, model_name="FNN")  
train_model(rnn_model, X_train, y_train, X_test, y_test, model_name="RNN")  
train_model(lstm_model, X_train, y_train, X_test, y_test, model_name="LSTM")  
train_model(cnn_model, X_train, y_train, X_test, y_test, model_name="CNN")  



[FNN] Epoch 0/100, Loss: 0.6337, Test Loss: 0.6239
[FNN] Epoch 10/100, Loss: 0.6267, Test Loss: 0.6169
[FNN] Epoch 20/100, Loss: 0.6194, Test Loss: 0.6092
[FNN] Epoch 30/100, Loss: 0.6102, Test Loss: 0.6000
[FNN] Epoch 40/100, Loss: 0.5985, Test Loss: 0.5884
[FNN] Epoch 50/100, Loss: 0.5832, Test Loss: 0.5737
[FNN] Epoch 60/100, Loss: 0.5639, Test Loss: 0.5553
[FNN] Epoch 70/100, Loss: 0.5396, Test Loss: 0.5328
[FNN] Epoch 80/100, Loss: 0.5093, Test Loss: 0.5048
[FNN] Epoch 90/100, Loss: 0.4729, Test Loss: 0.4715
[FNN] Training complete. Model & loss data saved.
[RNN] Epoch 0/100, Loss: 0.6590, Test Loss: 0.6562
[RNN] Epoch 10/100, Loss: 0.6393, Test Loss: 0.6365
[RNN] Epoch 20/100, Loss: 0.6206, Test Loss: 0.6181
[RNN] Epoch 30/100, Loss: 0.6021, Test Loss: 0.6000
[RNN] Epoch 40/100, Loss: 0.5834, Test Loss: 0.5818
[RNN] Epoch 50/100, Loss: 0.5645, Test Loss: 0.5634
[RNN] Epoch 60/100, Loss: 0.5453, Test Loss: 0.5448
[RNN] Epoch 70/100, Loss: 0.5260, Test Loss: 0.5261
[RNN] Epoch 80/1

In [5]:
data

,Axis,Unnamed: 1,Pe_results,Comp_results,TAC_Reading,Sober_classification,pid
0,x,0,0.977528,0.021814,0.088046,1,JR8022
1,x,1,0.966999,0.030278,0.176895,1,JR8022
2,x,2,0.961121,0.035584,0.206761,1,JR8022
3,y,0,0.932285,0.065238,0.088046,1,JR8022
4,y,1,0.979987,0.019418,0.176895,1,JR8022
...,...,...,...,...,...,...,...
481,z,12,0.961497,0.037951,0.134315,1,BK7610
482,z,13,0.987113,0.012408,0.148265,1,BK7610
483,z,14,0.956433,0.039753,0.160774,1,BK7610
484,z,15,0.915632,0.082337,0.166076,1,BK7610


In [10]:
import os
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import seaborn as sns

# ===================== Updated Prediction and Evaluation Functions ===================== #

def evaluate_model(model, X, y):
    """Evaluate model and return metrics"""
    with torch.no_grad():
        outputs = model(X)
        preds = torch.sigmoid(outputs) > 0.5
        y_np = y.cpu().numpy()
        preds_np = preds.cpu().numpy()
        
        acc = accuracy_score(y_np, preds_np)
        prec = precision_score(y_np, preds_np)
        rec = recall_score(y_np, preds_np)
        f1 = f1_score(y_np, preds_np)
        
    return {'Accuracy': acc, 'Precision': prec, 'Recall': rec, 'F1': f1}

def plot_time_series_comparison(original_data, predictions, model_name):
    """Plot original vs predicted time series data"""
    plt.figure(figsize=(15, 10))
    
    # Create time index if not exists
    if 'time_index' not in original_data.columns:
        original_data['time_index'] = np.arange(len(original_data))
        predictions['time_index'] = np.arange(len(predictions))
    
    # Plot 1: TAC Readings comparison
    plt.subplot(2, 2, 1)
    sns.lineplot(data=original_data, x='time_index', y='TAC_Reading', 
                 label='Original TAC', color='blue')
    sns.lineplot(data=predictions, x='time_index', y='TAC_Reading', 
                 label='Predicted TAC', color='red', alpha=0.7)
    plt.title('TAC Readings Comparison')
    plt.xlabel('Time Index')
    plt.ylabel('TAC Value')
    
    # Plot 2: Sober Classification comparison
    plt.subplot(2, 2, 2)
    sns.lineplot(data=original_data, x='time_index', y='Sober_classification', 
                 label='Original Sober', color='green')
    sns.lineplot(data=predictions, x='time_index', y='Predicted_Sober', 
                 label='Predicted Sober', color='orange', alpha=0.7)
    plt.title('Sober Classification Comparison')
    plt.xlabel('Time Index')
    plt.ylabel('Sober (1) / Not Sober (0)')
    
    # Plot 3: Feature comparison - PE Results
    plt.subplot(2, 2, 3)
    sns.lineplot(data=original_data, x='time_index', y='Pe_results', 
                 label='Original PE', color='purple')
    plt.title('Permutation Entropy Over Time')
    plt.xlabel('Time Index')
    plt.ylabel('PE Value')
    
    # Plot 4: Feature comparison - Complexity Results
    plt.subplot(2, 2, 4)
    sns.lineplot(data=original_data, x='time_index', y='Comp_results', 
                 label='Original Complexity', color='brown')
    plt.title('Complexity Over Time')
    plt.xlabel('Time Index')
    plt.ylabel('Complexity Value')
    
    plt.suptitle(f'Model: {model_name} - Time Series Analysis')
    plt.tight_layout()
    
    # Save plot
    os.makedirs("plots", exist_ok=True)
    plt.savefig(f'plots/{model_name}_time_series_comparison.png')
    plt.close()

def calculate_metrics(original_series, predicted_series):
    """Calculate complexity and permutation entropy metrics"""
    # Placeholder - implement your actual calculations here
    # For demonstration, we'll use simple statistical measures
    
    orig_complexity = np.std(original_series)  # Replace with actual complexity measure
    orig_pe = np.mean(original_series)        # Replace with actual PE calculation
    
    pred_complexity = np.std(predicted_series)
    pred_pe = np.mean(predicted_series)
    
    return {
        'Original_Complexity': orig_complexity,
        'Original_PE': orig_pe,
        'Predicted_Complexity': pred_complexity,
        'Predicted_PE': pred_pe
    }

# ===================== Main Prediction Pipeline ===================== #

def run_time_series_prediction(models_dict, data):
    """Run predictions for all models on the time series data"""
    # Prepare data
    X_all = data[['Pe_results', 'Comp_results', 'TAC_Reading']].values
    X_all = scaler.transform(X_all)
    X_all_tensor = torch.tensor(X_all, dtype=torch.float32).to(device)
    y_all_tensor = torch.tensor(data['Sober_classification'].values, 
                               dtype=torch.float32).view(-1, 1).to(device)
    
    results = {}
    metrics_comparison = []
    
    for model_name, model in models_dict.items():
        # Load model if not already loaded
        if isinstance(model, str):
            model_path = f"models/{model}.pth"
            model_class = globals()[model]()  # Get model class by name
            model_class.load_state_dict(torch.load(model_path))
            model_class.to(device)
            model = model_class
        
        # Evaluate on full dataset
        model_metrics = evaluate_model(model, X_all_tensor, y_all_tensor)
        results[model_name] = model_metrics
        
        # Make predictions
        with torch.no_grad():
            outputs = model(X_all_tensor)
            preds = torch.sigmoid(outputs) > 0.5
            preds = preds.cpu().numpy().flatten()
        
        # Create prediction dataframe
        pred_data = data.copy()
        pred_data['Predicted_Sober'] = preds
        
        # Calculate complexity and PE metrics
        tac_metrics = calculate_metrics(
            pred_data['TAC_Reading'][pred_data['Sober_classification'] == 1],
            pred_data['TAC_Reading'][pred_data['Predicted_Sober'] == 1]
        )
        
        # Store metrics for comparison
        metrics_comparison.append({
            'Model': model_name,
            **model_metrics,
            **tac_metrics
        })
        
        # Plot results
        plot_time_series_comparison(data, pred_data, model_name)
    
    # Save metrics and results
    pd.DataFrame(results).T.to_csv('logs/model_metrics.csv')
    pd.DataFrame(metrics_comparison).to_csv('logs/metrics_comparison.csv', index=False)
    
    # Print model comparison
    print("\nModel Performance Comparison:")
    print(pd.DataFrame(metrics_comparison).set_index('Model'))
    
    return pd.DataFrame(metrics_comparison)

# ===================== Run the Pipeline ===================== #

# Dictionary of models to evaluate
models_dict = {
    'FNN': fnn_model,
    'RNN': rnn_model,
    'LSTM': lstm_model,
    'CNN': cnn_model
}

# Run the prediction pipeline
results = run_time_series_prediction(models_dict, data)

# ===================== Additional Visualization ===================== #

def plot_metrics_comparison(metrics_df):
    """Plot comparison of model metrics"""
    plt.figure(figsize=(15, 10))
    
    # Plot performance metrics
    metrics_to_plot = ['Accuracy', 'Precision', 'Recall', 'F1']
    plt.subplot(2, 2, 1)
    metrics_df[metrics_to_plot].plot(kind='bar', ax=plt.gca())
    plt.title('Classification Metrics Comparison')
    plt.ylabel('Score')
    plt.xticks(rotation=45)
    
    # Plot complexity metrics
    plt.subplot(2, 2, 2)
    metrics_df[['Original_Complexity', 'Predicted_Complexity']].plot(kind='bar', ax=plt.gca())
    plt.title('Complexity Comparison')
    plt.ylabel('Complexity Value')
    plt.xticks(rotation=45)
    
    # Plot PE metrics
    plt.subplot(2, 2, 3)
    metrics_df[['Original_PE', 'Predicted_PE']].plot(kind='bar', ax=plt.gca())
    plt.title('Permutation Entropy Comparison')
    plt.ylabel('PE Value')
    plt.xticks(rotation=45)
    
    plt.tight_layout()
    plt.savefig('plots/metrics_comparison.png')
    plt.close()

# Plot the metrics comparison
plot_metrics_comparison(results.set_index('Model'))


Model Performance Comparison:
       Accuracy  Precision    Recall        F1  Original_Complexity  \
Model                                                                 
FNN    0.864865        1.0  0.522059  0.685990             0.037545   
RNN    0.792100        1.0  0.264706  0.418605             0.037545   
LSTM   0.785863        1.0  0.242647  0.390533             0.037545   
CNN    0.777547        1.0  0.213235  0.351515             0.037545   

       Original_PE  Predicted_Complexity  Predicted_PE  
Model                                                   
FNN       0.124173              0.028059      0.153862  
RNN       0.124173              0.019804      0.176093  
LSTM      0.124173              0.018726      0.178629  
CNN       0.124173              0.020578      0.178850  


In [11]:
import os
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import numpy as np
import pandas as pd
import torch

# ===================== Data Preparation ===================== #

# Load and prepare data
data = pd.read_csv("merged_data.csv")
data = data.dropna()

# Rename columns to handle case sensitivity
data = data.rename(columns={'pid': 'PID'})  # Ensure consistent column naming

# Verify PID column exists
if 'PID' not in data.columns:
    raise ValueError("PID column not found in data. Available columns: " + ", ".join(data.columns))

# Create time index for each PID group
data['time_index'] = data.groupby('PID').cumcount()

# ===================== Evaluation Functions ===================== #

def evaluate_model(model, X, y):
    """Evaluate model and return metrics"""
    with torch.no_grad():
        outputs = model(X)
        preds = torch.sigmoid(outputs) > 0.5
        y_np = y.cpu().numpy()
        preds_np = preds.cpu().numpy()
        
        acc = accuracy_score(y_np, preds_np)
        prec = precision_score(y_np, preds_np)
        rec = recall_score(y_np, preds_np)
        f1 = f1_score(y_np, preds_np)
        
    return {'Accuracy': acc, 'Precision': prec, 'Recall': rec, 'F1': f1}

# ===================== Visualization Functions ===================== #

def plot_pid_comparison(pid_data, model_name):
    """Plot comparison for a single PID"""
    plt.figure(figsize=(18, 12))
    
    # Plot 1: TAC Readings
    plt.subplot(3, 2, 1)
    plt.plot(pid_data['time_index'], pid_data['TAC_Reading'], 
             label='Original', color='blue')
    plt.plot(pid_data['time_index'][pid_data['Predicted_Sober'] == 1], 
             pid_data['TAC_Reading'][pid_data['Predicted_Sober'] == 1],
             'o', label='Predicted Sober', color='green', alpha=0.7)
    plt.plot(pid_data['time_index'][pid_data['Predicted_Sober'] == 0], 
             pid_data['TAC_Reading'][pid_data['Predicted_Sober'] == 0],
             'x', label='Predicted Not Sober', color='red', alpha=0.7)
    plt.title(f'TAC Readings for PID {pid_data["PID"].iloc[0]}')
    plt.xlabel('Time Index')
    plt.ylabel('TAC Value')
    plt.legend()
    
    # Plot 2: Sober Classification
    plt.subplot(3, 2, 2)
    plt.step(pid_data['time_index'], pid_data['Sober_classification'], 
             where='post', label='Original', color='blue')
    plt.step(pid_data['time_index'], pid_data['Predicted_Sober'], 
             where='post', label='Predicted', color='red', alpha=0.7)
    plt.title(f'Sober Classification for PID {pid_data["PID"].iloc[0]}')
    plt.xlabel('Time Index')
    plt.ylabel('Sober (1) / Not Sober (0)')
    plt.legend()
    
    # Plot 3: PE Results
    plt.subplot(3, 2, 3)
    plt.plot(pid_data['time_index'], pid_data['Pe_results'], 
             label='Original', color='purple')
    plt.title(f'Permutation Entropy for PID {pid_data["PID"].iloc[0]}')
    plt.xlabel('Time Index')
    plt.ylabel('PE Value')
    
    # Plot 4: Complexity Results
    plt.subplot(3, 2, 4)
    plt.plot(pid_data['time_index'], pid_data['Comp_results'], 
             label='Original', color='brown')
    plt.title(f'Complexity for PID {pid_data["PID"].iloc[0]}')
    plt.xlabel('Time Index')
    plt.ylabel('Complexity Value')
    
    # Plot 5: Sober vs TAC
    plt.subplot(3, 2, 5)
    plt.scatter(pid_data['TAC_Reading'], pid_data['Sober_classification'],
               label='Original', color='blue', alpha=0.5)
    plt.scatter(pid_data['TAC_Reading'], pid_data['Predicted_Sober'],
               label='Predicted', color='red', alpha=0.5)
    plt.title(f'TAC vs Sober Classification for PID {pid_data["PID"].iloc[0]}')
    plt.xlabel('TAC Reading')
    plt.ylabel('Sober Classification')
    plt.legend()
    
    plt.suptitle(f'Model: {model_name} - PID: {pid_data["PID"].iloc[0]}')
    plt.tight_layout()
    
    # Save plot
    os.makedirs("plots", exist_ok=True)
    plt.savefig(f'plots/{model_name}_PID_{pid_data["PID"].iloc[0]}.png')
    plt.close()

# ===================== Main Prediction Pipeline ===================== #

def run_pid_analysis(models_dict, data):
    """Run analysis for each PID and model"""
    # Prepare results storage
    pid_results = []
    model_metrics = []
    
    # Process each model
    for model_name, model in models_dict.items():
        # Load model if needed
        if isinstance(model, str):
            model_path = f"models/{model}.pth"
            model_class = globals()[model]()
            model_class.load_state_dict(torch.load(model_path))
            model_class.to(device)
            model = model_class
        
        # Prepare data
        X = data[['Pe_results', 'Comp_results', 'TAC_Reading']].values
        X = scaler.transform(X)
        X_tensor = torch.tensor(X, dtype=torch.float32).to(device)
        
        # Make predictions
        with torch.no_grad():
            outputs = model(X_tensor)
            preds = torch.sigmoid(outputs) > 0.5
            data['Predicted_Sober'] = preds.cpu().numpy().flatten()
        
        # Evaluate overall performance
        y_tensor = torch.tensor(data['Sober_classification'].values, 
                               dtype=torch.float32).view(-1, 1).to(device)
        overall_metrics = evaluate_model(model, X_tensor, y_tensor)
        model_metrics.append({'Model': model_name, **overall_metrics})
        
        # Analyze by PID
        for pid in data['PID'].unique():
            pid_data = data[data['PID'] == pid].copy()
            
            # Calculate metrics for this PID
            pid_metrics = {
                'Model': model_name,
                'PID': pid,
                'Accuracy': accuracy_score(pid_data['Sober_classification'], pid_data['Predicted_Sober']),
                'F1': f1_score(pid_data['Sober_classification'], pid_data['Predicted_Sober'])
            }
            pid_results.append(pid_metrics)
            
            # Create plots for this PID
            plot_pid_comparison(pid_data, model_name)
    
    # Save results
    pd.DataFrame(model_metrics).to_csv('logs/model_metrics.csv', index=False)
    pid_results_df = pd.DataFrame(pid_results)
    pid_results_df.to_csv('logs/pid_results.csv', index=False)
    
    # Create summary table
    summary_table = pid_results_df.pivot(index='Model', columns='PID', values='F1')
    summary_table['Average'] = pid_results_df.groupby('Model')['F1'].mean()
    
    print("\nModel Performance by PID (F1 Scores):")
    print(summary_table)
    
    return summary_table

# ===================== Run the Analysis ===================== #

# Dictionary of models to evaluate
models_dict = {
    'FNN': fnn_model,
    'RNN': rnn_model,
    'LSTM': lstm_model,
    'CNN': cnn_model
}

# Run the PID analysis
results_table = run_pid_analysis(models_dict, data)

# ===================== Additional Visualizations ===================== #

def plot_model_comparison(results_df):
    """Plot model comparison across PIDs"""
    plt.figure(figsize=(15, 8))
    
    # Melt the dataframe for seaborn
    melted_df = results_df.reset_index().melt(id_vars='Model', 
                                            value_vars=[col for col in results_df.columns if col != 'Average'],
                                            var_name='PID', value_name='F1 Score')
    
    # Plot F1 scores by model and PID
    sns.barplot(data=melted_df, x='PID', y='F1 Score', hue='Model')
    plt.title('Model Performance by PID')
    plt.xticks(rotation=45)
    plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.tight_layout()
    
    # Save plot
    plt.savefig('plots/model_comparison_by_pid.png')
    plt.close()

# Plot the comparison
plot_model_comparison(results_table)

/Users/mane/.pyenv/versions/3.11.5/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/mane/.pyenv/versions/3.11.5/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/mane/.pyenv/versions/3.11.5/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1509: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()


Model Performance by PID (F1 Scores):
PID      BK7610  BU4707    CC6740  DC6359  DK3500  HV0618  JB3156  JR8022  \
Model                                                                       
CNN    0.421053     0.0  0.500000     0.0     0.0     0.0     0.0     0.8   
FNN    0.666667     0.0  0.666667     1.0     0.0     0.8     0.0     0.8   
LSTM   0.461538     0.0  0.400000     0.0     0.0     0.0     0.0     0.8   
RNN    0.500000     0.0  0.588235     0.0     0.0     0.0     0.0     0.8   

PID      MC7070    MJ8002  PC6771    SA0297    SF3079   Average  
Model                                                            
CNN    0.125000  0.400000     0.0  0.363636  0.312500  0.224784  
FNN    0.888889  0.666667     0.0  0.875000  0.682927  0.542063  
LSTM   0.333333  0.400000     0.0  0.500000  0.363636  0.250654  
RNN    0.333333  0.400000     0.0  0.500000  0.363636  0.268093  
